# FZ Perfect Gas Example

This notebook demonstrates using FZ for parametric studies with the ideal gas law.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Funz/fz.github.io/blob/main/notebooks/perfectgas_example.ipynb)

## Install FZ

First, install FZ from GitHub:

In [ ]:
!pip install git+https://github.com/Funz/fz.git

## Create Input Template

Create an input file with variables and formulas:

In [ ]:
%%writefile input.txt
# input file for Perfect Gas Pressure
n_mol=$n_mol
T_kelvin=@($T_celsius + 273.15)
#@ def L_to_m3(L):
#@     return(L / 1000)
V_m3=@(L_to_m3($V_L))

## Create Calculation Script

Create a bash script to calculate pressure using the ideal gas law:

In [ ]:
%%writefile calculate.sh
#!/bin/bash

# Read input file
source $1

# Calculate pressure using ideal gas law
# P = nRT/V where R = 8.314 J/(mol·K)
pressure=$(echo "scale=4; $n_mol * 8.314 * $T_kelvin / $V_m3" | bc)

# Write output
echo "pressure = $pressure" > output.txt

echo "Done"

In [ ]:
!chmod +x calculate.sh

## Run Parametric Study

Use FZ to run all combinations of parameters:

In [ ]:
import fz

# Define the model
model = {
    "varprefix": "$",
    "formulaprefix": "@",
    "delim": "()",
    "commentline": "#",
    "output": {
        "pressure": "grep 'pressure = ' output.txt | awk '{print $3}'"
    }
}

# Define parameter values
input_variables = {
    "T_celsius": [10, 20, 30, 40],  # 4 temperatures
    "V_L": [1, 2, 5],                # 3 volumes
    "n_mol": 1.0                     # fixed amount
}

# Run all combinations (4 × 3 = 12 cases)
results = fz.fzr(
    "input.txt",
    input_variables,
    model,
    calculators="sh://bash calculate.sh",
    results_dir="results"
)

# Display results
print(f"\nCompleted {len(results)} calculations")
results

## Visualize Results

Plot pressure vs temperature for different volumes:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

for volume in sorted(results['V_L'].unique()):
    data = results[results['V_L'] == volume]
    plt.plot(data['T_celsius'], data['pressure'],
             marker='o', label=f'V={volume} L')

plt.xlabel('Temperature (°C)')
plt.ylabel('Pressure (Pa)')
plt.title('Ideal Gas: Pressure vs Temperature')
plt.legend()
plt.grid(True)
plt.show()

## Statistical Analysis

Analyze the results:

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(results[['T_celsius', 'V_L', 'pressure']].describe())

# Group by volume
print("\nAverage Pressure by Volume:")
print(results.groupby('V_L')['pressure'].mean())

## Download Results

Save and download the results:

In [ ]:
# Save to CSV
results.to_csv('perfectgas_results.csv', index=False)

# Download (in Colab)
from google.colab import files
files.download('perfectgas_results.csv')

## Next Steps

- Try modifying the parameter ranges
- Add more variables to study
- Explore [FZ plugins](https://funz.github.io/plugins/)
- Check out [more examples](https://funz.github.io/examples/)